# Australia Fires

Enlace a la documentación:
https://pypi.org/project/PyMySQL/

In [81]:
# Libraries
import pymysql.cursors
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

# Gathering and observing the Data

In [43]:
# Connect to the database
connection = pymysql.connect(host="34.229.137.164", port=25001, user="student", password="lebowsky", database="australia_fires_2", charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [44]:
query = 'SHOW TABLES'
# cursor.execute(query, connection)
pd.read_sql_query(query, connection)

,Tables_in_australia_fires_2
0,fire_archive_M6_96619
1,fire_archive_V1_96617
2,fire_nrt_M6_96619
3,fire_nrt_V1_96617
4,student_findings


# Table 0 : fire_archive_M6_96619

In [45]:
query = '''
SELECT *
FROM fire_archive_M6_96619
'''
tb0 = pd.read_sql_query(query, connection)
tb0

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,fire_type
0,0,-11.807,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
1,1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0
2,2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0
3,3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0
4,4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,MODIS,85,6.3,292.9,14.2,N,0
36007,36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,MODIS,27,6.3,290.9,6.4,N,0
36008,36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,MODIS,43,6.3,290.4,5.5,N,0
36009,36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,MODIS,41,6.3,290.3,5.7,N,0


Possible useful columns for training: brightness, scan, track, confidence, bright_t31, frp

In [46]:
# All of the data was gathered with the same instrument, we don't need it for training
tb0.groupby('instrument').count()

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight,fire_type
instrument,,,,,,,,,,,,,,,
MODIS,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011


In [47]:
tb0.groupby('satellite').count()

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,instrument,confidence,version,bright_t31,frp,daynight,fire_type
satellite,,,,,,,,,,,,,,,
Aqua,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541,20541
Terra,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470,15470


In [48]:
tb0.groupby('version').count()

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_t31,frp,daynight,fire_type
version,,,,,,,,,,,,,,,
6.3,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011,36011


In [49]:
# Useful columns for training and analysis
data0 = tb0.drop(['index', 'latitude', 'longitude', 'acq_date', 'acq_time', 'satellite', 'instrument', 'version', 'daynight'], axis=1)
data0

,brightness,scan,track,confidence,bright_t31,frp,fire_type
0,313.0,1.0,1.0,48,297.3,6.6,0
1,319.3,1.0,1.0,71,297.3,11.3,0
2,311.6,3.1,1.7,42,298.7,23.1,0
3,310.1,1.1,1.1,33,296.1,6.5,0
4,310.3,4.0,1.9,36,298.8,27.6,0
...,...,...,...,...,...,...,...
36006,312.7,1.2,1.1,85,292.9,14.2,0
36007,304.9,1.0,1.0,27,290.9,6.4,0
36008,301.7,1.0,1.0,43,290.4,5.5,0
36009,303.6,1.0,1.0,41,290.3,5.7,0


# Table 1: fire_archive_V1_96617

In [50]:
query = '''
SELECT *
FROM fire_archive_V1_96617
'''
tb1 = pd.read_sql_query(query, connection)
tb1
# This table does not have the daynight column

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,fire_type
0,0,-34.45902,150.88040,326.5,0.60,0.71,2019-08-01,246,N,VIIRS,n,1,290.7,5.8,3
1,1,-31.70724,151.07191,367.0,0.56,0.69,2019-08-01,247,N,VIIRS,h,1,291.9,10.4,0
2,2,-31.39626,149.65253,325.2,0.68,0.74,2019-08-01,247,N,VIIRS,n,1,292.7,5.6,0
3,3,-30.39774,152.06432,347.8,0.48,0.65,2019-08-01,247,N,VIIRS,n,1,287.6,9.1,0
4,4,-31.20248,151.89766,350.7,0.50,0.66,2019-08-01,247,N,VIIRS,n,1,292.3,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184773,184773,-32.37209,116.10032,296.1,0.48,0.40,2019-09-30,1702,N,VIIRS,n,1,284.8,1.2,0
184774,184774,-32.38958,116.11151,296.1,0.48,0.40,2019-09-30,1702,N,VIIRS,n,1,284.7,1.1,0
184775,184775,-33.12481,116.03968,299.9,0.47,0.40,2019-09-30,1702,N,VIIRS,n,1,285.0,1.3,0
184776,184776,-33.6348,122.92641,296.2,0.43,0.38,2019-09-30,1702,N,VIIRS,n,1,285.9,0.5,0


In [51]:
tb1.groupby('version').count()

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_ti5,frp,fire_type
version,,,,,,,,,,,,,,
1,184778,184778,184778,184778,184778,184778,184778,184778,184778,184778,184778,184778,184778,184778


In [52]:
tb1.groupby('fire_type').count()

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp
fire_type,,,,,,,,,,,,,,
0,180150,180150,180150,180150,180150,180150,180150,180150,180150,180150,180150,180150,180150,180150
2,1893,1893,1893,1893,1893,1893,1893,1893,1893,1893,1893,1893,1893,1893
3,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735


In [53]:
tb1.groupby('confidence').count()

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,version,bright_ti5,frp,fire_type
confidence,,,,,,,,,,,,,,
h,13597,13597,13597,13597,13597,13597,13597,13597,13597,13597,13597,13597,13597,13597
l,13256,13256,13256,13256,13256,13256,13256,13256,13256,13256,13256,13256,13256,13256
n,157925,157925,157925,157925,157925,157925,157925,157925,157925,157925,157925,157925,157925,157925


In [54]:
# Useful columns for training and analysis
data1 = tb1.drop(['index', 'latitude', 'longitude', 'acq_date', 'acq_time', 'satellite', 'instrument', 'version'], axis=1)
data1

,bright_ti4,scan,track,confidence,bright_ti5,frp,fire_type
0,326.5,0.60,0.71,n,290.7,5.8,3
1,367.0,0.56,0.69,h,291.9,10.4,0
2,325.2,0.68,0.74,n,292.7,5.6,0
3,347.8,0.48,0.65,n,287.6,9.1,0
4,350.7,0.50,0.66,n,292.3,9.4,0
...,...,...,...,...,...,...,...
184773,296.1,0.48,0.40,n,284.8,1.2,0
184774,296.1,0.48,0.40,n,284.7,1.1,0
184775,299.9,0.47,0.40,n,285.0,1.3,0
184776,296.2,0.43,0.38,n,285.9,0.5,0


In [55]:
# We also have to convert the confidence column to numerical, lets use LabelEncoder
# Create a LabelEncoder object
le = LabelEncoder()
le.fit(data1['confidence'])
print('Classes:', le.classes_)
print('Converted confidence column:', le.transform(data1['confidence']))
# Changing the column in the dataframe
data1['confidence'] = le.transform(data1['confidence'])

Classes: ['h' 'l' 'n']
Converted confidence column: [2 0 2 ... 2 2 2]


In [56]:
data1

,bright_ti4,scan,track,confidence,bright_ti5,frp,fire_type
0,326.5,0.60,0.71,2,290.7,5.8,3
1,367.0,0.56,0.69,0,291.9,10.4,0
2,325.2,0.68,0.74,2,292.7,5.6,0
3,347.8,0.48,0.65,2,287.6,9.1,0
4,350.7,0.50,0.66,2,292.3,9.4,0
...,...,...,...,...,...,...,...
184773,296.1,0.48,0.40,2,284.8,1.2,0
184774,296.1,0.48,0.40,2,284.7,1.1,0
184775,299.9,0.47,0.40,2,285.0,1.3,0
184776,296.2,0.43,0.38,2,285.9,0.5,0


# Table 2 : fire_nrt_M6_96619

In [57]:
query = '''
SELECT *
FROM fire_nrt_M6_96619
'''
tb2 = pd.read_sql_query(query, connection)
tb2
# This table does not have the fire_type column (We have to predict it)

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,0,-14.281,143.636,323.9,1.7,1.3,2019-10-01,25,Terra,MODIS,70,6.0NRT,302.3,26.8,D
1,1,-14.284,143.532,343.5,1.7,1.3,2019-10-01,25,Terra,MODIS,90,6.0NRT,306.3,84.3,D
2,2,-14.302,143.706,320.2,1.7,1.3,2019-10-01,25,Terra,MODIS,30,6.0NRT,305.0,14.1,D
3,3,-14.283,143.652,320.4,1.7,1.3,2019-10-01,25,Terra,MODIS,57,6.0NRT,303.3,18.4,D
4,4,-14.285,143.521,349.4,1.7,1.3,2019-10-01,25,Terra,MODIS,94,6.0NRT,304.7,110.7,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183588,183588,-32.675,122.113,300.5,2.4,1.5,2020-01-11,1515,Terra,MODIS,19,6.0NRT,289.3,15.3,N
183589,183589,-16.708,129.646,300.5,1.0,1.0,2020-01-11,1645,Aqua,MODIS,16,6.0NRT,280.8,2.9,N
183590,183590,-16.747,129.653,301.8,1.0,1.0,2020-01-11,1645,Aqua,MODIS,23,6.0NRT,277.9,3.7,N
183591,183591,-32.484,121.319,306.5,1.4,1.2,2020-01-11,1650,Aqua,MODIS,68,6.0NRT,291.9,11.6,N


In [58]:
tb2.groupby('version').count()

,index,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_t31,frp,daynight
version,,,,,,,,,,,,,,
6.0NRT,183593,183593,183593,183593,183593,183593,183593,183593,183593,183593,183593,183593,183593,183593


In [59]:
# Useful columns for training and analysis
data2 = tb2.drop(['index', 'latitude', 'longitude', 'acq_date', 'acq_time', 'satellite', 'instrument', 'version', 'daynight'], axis=1)
data2

,brightness,scan,track,confidence,bright_t31,frp
0,323.9,1.7,1.3,70,302.3,26.8
1,343.5,1.7,1.3,90,306.3,84.3
2,320.2,1.7,1.3,30,305.0,14.1
3,320.4,1.7,1.3,57,303.3,18.4
4,349.4,1.7,1.3,94,304.7,110.7
...,...,...,...,...,...,...
183588,300.5,2.4,1.5,19,289.3,15.3
183589,300.5,1.0,1.0,16,280.8,2.9
183590,301.8,1.0,1.0,23,277.9,3.7
183591,306.5,1.4,1.2,68,291.9,11.6


# Table 3: fire_nrt_V1_96617

In [60]:
query = '''
SELECT *
FROM fire_nrt_V1_96617
'''
tb3 = pd.read_sql_query(query, connection)
tb3
# This table does not have the fire_type column (We have to predict it)

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,0,-42.69706,147.70634,333.6,0.57,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,293.4,4.0,D
1,1,-42.26889,147.31104,336.2,0.39,0.44,2019-10-01,336,N,VIIRS,n,1.0NRT,296.9,3.9,D
2,2,-40.85202,145.38068,327.9,0.46,0.47,2019-10-01,336,N,VIIRS,n,1.0NRT,295.3,3.7,D
3,3,-42.39329,147.47144,346.9,0.38,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,294.2,6.2,D
4,4,-42.69701,147.70584,334.1,0.56,0.43,2019-10-01,336,N,VIIRS,n,1.0NRT,293.5,4.1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956252,956252,-32.66628,122.15253,306.9,0.39,0.44,2020-01-11,1630,N,VIIRS,n,1.0NRT,288.6,0.8,N
956253,956253,-32.58616,123.39582,299.0,0.51,0.41,2020-01-11,1630,N,VIIRS,n,1.0NRT,287.4,0.7,N
956254,956254,-33.37853,115.94735,309.7,0.40,0.60,2020-01-11,1630,N,VIIRS,n,1.0NRT,291.7,1.0,N
956255,956255,-33.21827,115.75078,300.2,0.49,0.65,2020-01-11,1806,N,VIIRS,n,1.0NRT,290.2,1.1,N


In [61]:
tb3.groupby('version').count()

,index,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_ti5,frp,daynight
version,,,,,,,,,,,,,,
1.0NRT,956257,956257,956257,956257,956257,956257,956257,956257,956257,956257,956257,956257,956257,956257


In [62]:
# Useful columns for training and analysis
data3 = tb3.drop(['index', 'latitude', 'longitude', 'acq_date', 'acq_time', 'satellite', 'instrument', 'version', 'daynight'], axis=1)
data3

,bright_ti4,scan,track,confidence,bright_ti5,frp
0,333.6,0.57,0.43,n,293.4,4.0
1,336.2,0.39,0.44,n,296.9,3.9
2,327.9,0.46,0.47,n,295.3,3.7
3,346.9,0.38,0.43,n,294.2,6.2
4,334.1,0.56,0.43,n,293.5,4.1
...,...,...,...,...,...,...
956252,306.9,0.39,0.44,n,288.6,0.8
956253,299.0,0.51,0.41,n,287.4,0.7
956254,309.7,0.40,0.60,n,291.7,1.0
956255,300.2,0.49,0.65,n,290.2,1.1


In [63]:
# We also have to convert the confidence column to numerical, lets use LabelEncoder
# Create a LabelEncoder object
le = LabelEncoder()
le.fit(data3['confidence'])
print('Classes:', le.classes_)
print('Converted confidence column:', le.transform(data3['confidence']))
# Changing the column in the dataframe
data3['confidence'] = le.transform(data3['confidence'])

Classes: ['h' 'l' 'n']
Converted confidence column: [2 2 2 ... 2 2 2]


# Table 4: student_findings

In [64]:
query = '''
SELECT *
FROM student_findings
'''
tb4 = pd.read_sql_query(query, connection)
tb4
# This table is empty, we can ignore it

,index,chosen_model,acc_score,recall_score


From the observations realized above on the databases and the columns, I have selected the columns I consider will be useful for the predictions. According to the structure of the tables themselves, table0 will be used as training data to predict table2, because they share the same structure and table1 will be used as training data to predict table3, because they also share the same structure.

# Defining Variables and Models

## Prediction 1: M6_96619 fire data

In [65]:
data0

,brightness,scan,track,confidence,bright_t31,frp,fire_type
0,313.0,1.0,1.0,48,297.3,6.6,0
1,319.3,1.0,1.0,71,297.3,11.3,0
2,311.6,3.1,1.7,42,298.7,23.1,0
3,310.1,1.1,1.1,33,296.1,6.5,0
4,310.3,4.0,1.9,36,298.8,27.6,0
...,...,...,...,...,...,...,...
36006,312.7,1.2,1.1,85,292.9,14.2,0
36007,304.9,1.0,1.0,27,290.9,6.4,0
36008,301.7,1.0,1.0,43,290.4,5.5,0
36009,303.6,1.0,1.0,41,290.3,5.7,0


In [66]:
data0.groupby('fire_type').count()

,brightness,scan,track,confidence,bright_t31,frp
fire_type,,,,,,
0,35666,35666,35666,35666,35666,35666
2,335,335,335,335,335,335
3,10,10,10,10,10,10


In [67]:
# Variables
X_train1 = np.array(data0.loc[:, :'frp'])
y_train1 = np.array(data0['fire_type'])
X_test1 = np.array(data2.loc[:, :'frp'])

In [68]:
# # Splitting data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [69]:
# Logistic Regression Model
from sklearn import linear_model
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train1, y_train1)
# Predictions
y_pred1 = log_reg.predict(X_test1)
y_pred1

array([0, 0, 0, ..., 0, 0, 0])

In [95]:
# Uniting all results in a dictionary to create a df later
predictions1 = {'log_reg': y_pred1}
predictions1

{'log_reg': array([0, 0, 0, ..., 0, 0, 0])}

In [70]:
# Score of our model
log_reg.score(X_train1, y_train1)

0.9903918247202244

In [96]:
# Random Forest Classifier Model
from sklearn.ensemble import RandomForestClassifier
rand_for = RandomForestClassifier()
rand_for.fit(X_train1, y_train1)
# Predictions
y_pred1 = rand_for.predict(X_test1)
predictions1['rand_for'] = y_pred1
y_pred1

array([0, 0, 0, ..., 0, 0, 0])

In [77]:
# Score of our model
rand_for.score(X_train1, y_train1)

1.0

In [97]:
# XGBoost 
model = XGBClassifier() 
model.fit(X_train1, y_train1) 
#Make predictions for test data 
y_pred1 = model.predict(X_test1)
predictions1['xgboost'] = y_pred1
y_pred1

[14:51:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0, 0, 0, ..., 0, 0, 0])

In [84]:
# Score of our model
model.score(X_train1, y_train1)

0.993696370553442

In [98]:
pd.DataFrame(predictions1)

,log_reg,rand_for,xgboost
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
183588,0,0,0
183589,0,0,0
183590,0,0,0
183591,0,0,0


## Prediction 2: V1_96617 fire data

In [71]:
# Variables
X_train2 = np.array(data1.loc[:, :'frp'])
y_train2 = np.array(data1['fire_type'])
X_test2 = np.array(data3.loc[:, :'frp'])

In [72]:
# Logistic Regression Model
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train2, y_train2)
# Predictions
y_pred2 = log_reg.predict(X_test2)
y_pred2

LogisticRegression()

In [90]:
# Uniting all results in a dictionary to create a df later
predictions2 = {'log_reg': y_pred2}
predictions2

{'log_reg': array([0, 0, 0, ..., 0, 0, 0])}

In [74]:
# Score of our model
log_reg.score(X_train2, y_train2)

0.974953728257693

In [91]:
# Random Forest Classifier Model
rand_for = RandomForestClassifier()
rand_for.fit(X_train2, y_train2)
# Predictions
y_pred2 = rand_for.predict(X_test2)
predictions2['rand_for'] = y_pred2
y_pred2

array([0, 0, 0, ..., 0, 0, 0])

In [93]:
# XGBoost 
model = XGBClassifier() 
model.fit(X_train2, y_train2) 
#Make predictions for test data 
y_pred2 = model.predict(X_test2)
predictions2['xgboost'] = y_pred2
y_pred2

[14:48:49] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0, 0, 0, ..., 0, 0, 0])

In [86]:
# Score of our model
model.score(X_train1, y_train1)

0.9904195940129404

In [94]:
pd.DataFrame(predictions2)

,log_reg,rand_for,xgboost
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
956252,0,0,0
956253,0,0,0
956254,0,0,0
956255,0,0,0
